# Module: Fetch Data
*get data from TSETMC and directly write to db*

## Prepearing Classification Data

In [1]:
import pandas as pd
import requests
import zeep
import json
import pytse_client

from utils import debugging_log, data_log, ar_to_fa_series, ar_to_fa

In [2]:
pd.set_option('display.max_columns', 30)

## Definition & Assignment

In [3]:
request_headers = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Referer': 'http://main.tsetmc.com/StaticContent/WebServiceHelp'
}
cookie_jar = {'ASP.NET_SessionId': 'wa40en1alwxzjnqehjntrv5j'}

IDENTITY_URL = "http://cdn.tsetmc.com/api/Instrument/GetInstrumentIdentity/{}"
test_ids = ['46348559193224090','20024911381434086','22129017544200','94378630649293']

In [4]:
def last_possible_deven():
    client = zeep.Client(
        'http://service.tsetmc.com/WebService/TseClient.asmx?wsdl')
    last_workday = client.service.LastPossibleDeven()
    return last_workday.split(';')[0] if last_workday.split(
        ';')[1] == last_workday.split(';')[0] else print(
            last_workday)  # for debuging purposes

In [5]:
def InstrumentAndShare(last_fetch_date, last_record_id):
    client = zeep.Client(
        wsdl='http://service.tsetmc.com/WebService/TseClient.asmx?wsdl')

    instruments, share_increase = client.service.InstrumentAndShare(
        last_fetch_date, last_record_id).split('@')

    if instruments:
        instruments = instruments.split(';')
        instruments = pd.DataFrame(
            [instrument.split(',') for instrument in instruments])
        instruments.columns = [
            'id', 'inst_code_12', 'inst_code_5', 'name_latin',
            'company_code_4', 'ticker', 'name', 'isin', 'mod_date',
            'market_code', 'company_name', 'status_code', 'group_code',
            'market_type_code', 'tableu_code', 'industry_sector_code',
            'industry_subsector_code', 'type_code'
        ]
        instruments.set_index('id', inplace=True)
        instruments['ticker'] = ar_to_fa_series(instruments['ticker'])
        instruments['name'] = ar_to_fa_series(instruments['name'])
        instruments['company_name'] = ar_to_fa_series(
            instruments['company_name'])

    if share_increase:
        share_increase = share_increase.split(';')
        share_increase = pd.DataFrame(
            [share.split(',') for share in share_increase])
        share_increase.columns = [
            'record_id', 'id', 'date', 'before_raise', 'after_raise'
        ]
        #capital_increase['record_id'] = capital_increase['record_id'].astype(int)
        share_increase.set_index('record_id', inplace=True)
    else:
        share_increase = None

    return instruments, share_increase


In [6]:
def flatten_json(y):
    out = {}
    
    def flatten(x, name =''):
        
        # If the Nested key-value 
        # pair is of dict type
        if type(x) is dict:
            
            for a in x:
                flatten(x[a], name + a + '_')
                
        # If the Nested key-value
        # pair is of list type
        elif type(x) is list:
            
            i = 0
            
            for a in x:                
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x
        
    flatten(y)
    return out

In [7]:
def get_identity(id):
    data_log.info(f'Going for Identity of "{id}"')
    result = requests.get(
        IDENTITY_URL.format(id),
        headers=request_headers,
        cookies=cookie_jar)
    result.raise_for_status() #raises exception when not a 2xx response
    result = json.loads(result.text)
    data_log.info(f'Identity of "{id}" recieved correctly.')
    return flatten_json(result['instrumentIdentity'])

In [8]:
last_date = last_possible_deven()
insts, shares = InstrumentAndShare(0,0)
print(last_possible_deven())

20230904


## Initialize Identity

In [9]:
data_log.info('|||||||||||||||||||||||||||||| Identity initialization')
all_identities = pd.DataFrame({index:get_identity(index) for index in insts[insts['tableu_code'] != '6'].index}).transpose()
# اینبار تو یک دق و ۲۵ ثانیه تموم شد! و اینکه تا قبل از این ساعت یعنی ۸ و ۴۹ شب ارور میداد ک ریسپانس خالی میگرفت.
# شاید از ی ساعتی ببعد ترافیک سرور کم میشه و راحت میشه دیتا گرفت. بعدا چک کنم ک مطمئن بشم
# Update: without proxy (even with bypassing tsetmc.com) works better.
data_log.info('|||||||||||||||||||||||||||||| Identity loaded succesfuly')
all_identities.index.name = 'insCode'

In [14]:
from datetime import datetime

currentDateTime = datetime.now().strftime("%m-%d-%Y %H-%M-%S %p")
all_identities.to_csv(f"identities {currentDateTime}.csv")

## Categorization

In [16]:
allidz = pd.read_csv('identities 09-04-2023 22-51-38 PM.csv', index_col='insCode')
with pd.option_context('display.max_rows', 3):
    display(allidz)

,sector_dEven,sector_cSecVal,sector_lSecVal,subSector_dEven,subSector_cSecVal,subSector_cSoSecVal,subSector_lSoSecVal,cValMne,lVal18,cSocCSAC,lSoc30,yMarNSC,yVal,insCode.1,lVal30,lVal18AFC,flow,cIsin,zTitad,baseVol,instrumentID,cgrValCot,cComVal,lastDate,sourceID,flowTitle,cgrValCotTitle
insCode,,,,,,,,,,,,,,,,,,,,,,,,,,,
22129017544200,0,34,خودرو و ساخت قطعات,0,NaN,3499,ساير قطعات يدكي و وسايل نقليه موتوري,AZIX1,Saipa Azin-R,AZIN,سايپاآذين‌,NO,400,0,ح . سايپاآذين‌,خاذينح,0,IRR1AZIN0106,0.0,0,IRR1AZIN0101,N2,5,0,0,NaN,بازار دوم بورس
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72044846109864381,0,25,لاستيك و پلاستيك,0,NaN,2511,توليد تاير و بازسازي تايرهاي لاستيكي,ARTZ1,Artavil Tire,ARTZ,مجتمع صنعتي آرتاويل تاير,NO,303,0,مجتمع صنعتي آرتاويل تاير,پارتا,0,IRO3ARTZ0005,0.0,0,IRO3ARTZ0001,Z1,3,0,0,NaN,بازار دوم فرابورس


In [17]:
import ipywidgets as widgets

In [68]:
sector_combo_box = widgets.Combobox(
    # value='John',
    placeholder='Choose Industry Group',
    options=tuple(allidz['sector_lSecVal'].unique()),
    description='Industry Combobox:',
    ensure_option=True,
    disabled=False
)
sub_sector_dropdown = widgets.Dropdown(
    options=allidz['sector_lSecVal'].unique(),
    #value='2',
    description='SubSector:',
    disabled=False,
)

In [69]:
def show_tickers_of_industry(industry):
    display(allidz[allidz['sector_lSecVal'] == industry][['lSoc30','lVal18AFC','subSector_lSoSecVal']])


In [70]:

widgets.interact(show_tickers_of_industry, industry=sub_sector_dropdown)

interactive(children=(Dropdown(description='SubSector:', options=('خودرو و ساخت قطعات', 'اوراق تامين مالي', 'س…

<function __main__.show_tickers_of_industry(industry)>

In [ ]:
com

In [30]:
a = widgets.IntSlider(description="Delayed", continuous_update=False)
b = widgets.IntText(description="Delayed", continuous_update=False)
c = widgets.IntSlider(description="Continuous", continuous_update=True)
d = widgets.IntText(description="Continuous", continuous_update=True)

widgets.link((a, 'value'), (b, 'value'))
widgets.link((a, 'value'), (c, 'value'))
widgets.link((a, 'value'), (d, 'value'))
widgets.HBox([a,b,c,d])

In [17]:
allidz[allidz['sector_lSecVal'] == 'خودرو و ساخت قطعات']

,insCode,sector_dEven,sector_cSecVal,sector_lSecVal,subSector_dEven,subSector_cSecVal,subSector_cSoSecVal,subSector_lSoSecVal,cValMne,lVal18,cSocCSAC,lSoc30,yMarNSC,yVal,insCode.1,lVal30,lVal18AFC,flow,cIsin,zTitad,baseVol,instrumentID,cgrValCot,cComVal,lastDate,sourceID,flowTitle,cgrValCotTitle
0,22129017544200,0,34,خودرو و ساخت قطعات,0,NaN,3499,ساير قطعات يدكي و وسايل نقليه موتوري,AZIX1,Saipa Azin-R,AZIN,سايپاآذين‌,NO,400,0,ح . سايپاآذين‌,خاذينح,0,IRR1AZIN0106,0.0,0,IRR1AZIN0101,N2,5,0,0,NaN,بازار دوم بورس
18,590761101519570,0,34,خودرو و ساخت قطعات,0,NaN,3410,توليد وسايل نقليه موتوري,SIPX1,Saipa-R,SIPA,سايپا,NO,400,0,ح . سايپا,خساپاح,0,IRR1SIPA0103,0.0,0,IRR1SIPA0101,N2,5,0,0,NaN,بازار دوم بورس
47,1852132120586484,0,34,خودرو و ساخت قطعات,0,NaN,3430,قطعات يدكي و جانبي وسايل نقليه موتوري,INDX1,Indamin-R,INDM,كمك‌فنرايندامين‌,NO,400,0,ح . كمك‌فنرايندامين‌,خكمكح,0,IRR1INDM0108,0.0,0,IRR1INDM0101,N2,5,0,0,NaN,بازار دوم بورس
58,2262715289665960,0,34,خودرو و ساخت قطعات,0,NaN,3430,قطعات يدكي و جانبي وسايل نقليه موتوري,RADX1,Iran Radiator-R,RADI,رادياتور ايران‌,NO,400,0,ح . رادياتور ايران‌,ختورح,0,IRR1RADI0108,0.0,0,IRR1RADI0101,N1,3,0,0,NaN,بازار اول (تابلوی فرعی) بورس
69,2589887561569709,0,34,خودرو و ساخت قطعات,0,NaN,3410,توليد وسايل نقليه موتوري,ZMYD1,Zamyad,ZMYD,زامياد,NO,300,0,زامياد,خزاميا,0,IRO1ZMYD0002,0.0,0,IRO1ZMYD0001,N2,5,0,0,NaN,بازار دوم بورس
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1950,64842837716888827,0,34,خودرو و ساخت قطعات,0,NaN,3410,توليد وسايل نقليه موتوري,SDLP1,S*Saipa Diesel,SDLP,سايپا ديزل,NO,309,0,سايپا ديزل,خكاوه,0,IRO7SDLP0007,0.0,0,IRO7SDLP0001,P1,7,0,0,NaN,بازار پایه زرد فرابورس
1987,65883838195688438,0,34,خودرو و ساخت قطعات,0,NaN,3410,توليد وسايل نقليه موتوري,IKCO1,Iran Khodro,IKCO,ايران‌ خودرو,NO,300,0,ايران‌ خودرو,خودرو,0,IRO1IKCO0008,0.0,0,IRO1IKCO0001,N2,5,0,0,NaN,بازار دوم بورس
1998,66127247173352975,0,34,خودرو و ساخت قطعات,0,NaN,3430,قطعات يدكي و جانبي وسايل نقليه موتوري,ESHP1,Ashtad Iran,ESHP,صنايع توليدي اشتاد ايران‌,NO,309,0,صنايع توليدي اشتاد ايران‌,تشتاد,0,IRO7ESHP0002,0.0,0,IRO7ESHP0001,P1,7,0,0,NaN,بازار پایه زرد فرابورس
2133,70289374539527245,0,34,خودرو و ساخت قطعات,0,NaN,3430,قطعات يدكي و جانبي وسايل نقليه موتوري,GHAT1,Iran Auto-Parts,GHAT,قطعات‌ اتومبيل‌ ايران‌,NO,300,0,قطعات‌ اتومبيل‌ ايران‌,ختوقا,0,IRO1GHAT0002,0.0,0,IRO1GHAT0001,N1,1,0,0,NaN,بازار اول (تابلوی اصلی) بورس
